# Datashading LandSat8 raster satellite imagery

Datashader is fundamentally a rasterizing library, turning data into rasters (image-like arrays), but it is also useful for already-rasterized data like satellite imagery.  For raster data, datashader uses the separate [xarray](http://xarray.pydata.org/) library to re-render the data to whatever new bounding box and resolution the user requests, and the rest of the datashader pipeline can then be used to visualize and analyze the data.  This demo shows how to work with a set of raster satellite data, generating images as needed and overlaying them on geographic coordinates.

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import geoviews as gv
import datashader as ds
import cartopy.crs as ccrs
MERCATOR = ccrs.GOOGLE_MERCATOR

from holoviews.operation.datashader import datashade, regrid, shade
from bokeh.tile_providers import STAMEN_TONER

hv.extension('bokeh', width=80)

### Load LandSat Data 

LandSat data is measured in different frequency bands, revealing different types of information:

```
Band      Wavelength     Resolution   Description
          (micrometers)  (meters)

Band 1     0.43 - 0.45     30         Coastal aerosol
Band 2     0.45 - 0.51     30         Blue
Band 3     0.53 - 0.59     30         Green
Band 4     0.64 - 0.67     30         Red
Band 5     0.85 - 0.88     30         Near Infrared (NIR)
Band 6     1.57 - 1.65     30         SWIR 1
Band 7     2.11 - 2.29     30         SWIR 2
Band 8     0.50 - 0.68     15         Panchromatic
Band 9     1.36 - 1.38     30         Cirrus
Band 10   10.60 - 11.19   100 * (30)  Thermal Infrared (TIRS) 1
Band 11   11.50 - 12.51   100 * (30)  Thermal Infrared (TIRS) 2
```

In [ ]:
file_path = './data/MERCATOR_LC80210392016114LGN00_B%s.TIF'
bands = list(range(1, 12)) + ['QA']
bands = [xr.open_rasterio(file_path%band).load()[0] for band in bands]
band_spectra = [0.44, 0.48, 0.56, 0.655, 0.865, 1.61, 2.2, 0.59, 1.37, 10.895, 12.005]

## Rendering LandSat data as images

The bands measured by LandSat include wavelengths covering the visible spectrum, but also other ranges, and so it's possible to visualize this data in many different ways, in both true color (using the visible spectrum directly) or false color (usually showing other bands).  Some examples are shown in the sections below.

### Just the Blue Band

Using datashader's default histogram-equalized colormapping, the full range of data is visible in the plot:

In [ ]:
%opts RGB [width=600 height=600]
tiles = gv.WMTS(STAMEN_TONER)
tiles * shade(regrid(hv.Image(bands[1])), cmap=['black', 'white']).redim(x='Longitude', y='Latitude')

You will usually want to zoom in, which will re-rasterize the image if you are in a live notebook, and then re-equalize the colormap to show all the detail available.  If you are on a static copy of the notebook, only the original resolution at which the image was rendered will be available, but zooming will still update the map tiles to whatever resolution is requested.

The plots below use a different type of colormap processing, implemented as a custom transfer function:

In [ ]:
from datashader.utils import ngjit
nodata= 1

@ngjit
def normalize_data(agg):
    out = np.zeros_like(agg)
    min_val = 0
    max_val = 2**16 - 1
    range_val = max_val - min_val
    col, rows = agg.shape
    c = 40
    th = .125
    for x in range(col):
        for y in range(rows):
            val = agg[x, y]
            norm = (val - min_val) / range_val
            norm = 1 / (1 + np.exp(c * (th - norm))) # bonus
            out[x, y] = norm * 255.0
    return out

def combine_bands(r, g, b):
    xs, ys = r['x'], r['y']
    r, g, b = [ds.utils.orient_array(img) for img in (r, g, b)]
    a = (np.where(np.logical_or(np.isnan(r),r<=nodata),0,255)).astype(np.uint8)    
    r = (normalize_data(r)).astype(np.uint8)
    g = (normalize_data(g)).astype(np.uint8)
    b = (normalize_data(b)).astype(np.uint8)
    col, rows = r.shape
    return hv.RGB((xs, ys[::-1], r, g, b, a), vdims=list('RGBA'))

### True Color (Red=Red, Green=Green, Blue=Blue)

Mapping the Red, Green, and Blue bands to the R, G, and B channels of an image reconstructs the image as it would appear to an ordinary camera from that viewpoint:

In [ ]:
true_color = combine_bands(bands[3], bands[2], bands[1])
tiles * regrid(true_color)

Other combinations highlight particular features of interest based on the different spectral properties of reflectances from various objects and surfaces:

### Color Infrared (Vegetation) (Red=Near Infrared, Green=Red, Blue=Green)

In [ ]:
vegetation = combine_bands(bands[4], bands[3], bands[2])
tiles * regrid(vegetation)

### False Color (Urban) (Red=SWIR 2, Green=SWIR 1, Blue=Red)

In [ ]:
urban = combine_bands(bands[6], bands[5], bands[3])
tiles * regrid(urban)

### False Color 2 (Red=Near Infrared, Green=SWIR 1, Blue=Coastal)

In [ ]:
false_color = combine_bands(bands[4], bands[6], bands[0])
tiles * regrid(false_color)

### Land vs. Water (Red=Near Infrared, Green=SWIR 1, Blue=Red)

In [ ]:
land_vs_water = combine_bands(bands[4], bands[5], bands[3])
tiles * regrid(land_vs_water)

### Shortwave Infrared (Red=SWIR2, Green=Near Infrared, Blue=Red)

In [ ]:
infrared = combine_bands(bands[6], bands[4], bands[3])
tiles * regrid(infrared)

All the various ways of combining aggregates supported by [xarray](http://xarray.pydata.org) are available for these channels, making it simple to make your own custom visualizations highlighting any combination of bands that reveal something of interest.

### Revealing the spectrum

In [ ]:
%%opts RGB Curve [width=500 height=500] Curve (color='black') [logx=True]
def get_spectrum(x, y):
    if x is None or y is None:
        spectrum = np.zeros(11)
    else:
        spectrum = band_map.sample(x=x, y=y)['z'][:-1]
    point = gv.Points([(x, y)], crs=MERCATOR)
    point = gv.operation.project_points(point, projection=ccrs.PlateCarree())
    label = 'Lon: %.3f, Lat: %.3f' % tuple(point.array()[0])
    return hv.Curve((band_spectra, spectrum), label=label,
                    kdims=['Wavelength (µm)'], vdims=['Luminance']).sort()

band_map = hv.HoloMap({i: hv.Image(band) for i, band in enumerate(bands)})
tap = hv.streams.PointerXY(source=true_color)
spectrum = hv.DynamicMap(get_spectrum, streams=[tap]).redim.range(Luminance=(0, 30000))

tiles * regrid(true_color) + spectrum